In [31]:
pip install import-ipynb

SyntaxError: invalid syntax (2252928269.py, line 1)

In [32]:
import import_ipynb

In [84]:
import Email
import NER
import Scheduling
import Training

In [37]:
# import pandas as pd
# import re
# import torch
# import numpy as np

# import transformers
# from transformers import BertTokenizer
# from torch import nn
# from transformers import BertModel
# from torch.optim import Adam
# from tqdm import tqdm
# import nltk
# nltk.download('punkt')
# from nltk.tag.stanford import StanfordNERTagger
# from nltk.tokenize import word_tokenize
# from sklearn import metrics


ModuleNotFoundError: No module named 'regex'

  GET EMAIL AND RELEVANT DETAILS

In [ ]:
#get the service and extract email components
service = Email.getService()
email_details = Email.getEmails(service)
userInput = email_details["Message"]
print(userInput)

Install neccessary packages for training and evaluating the BERT classifier 

In [123]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 27.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.5/167.5 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 43.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.1/160.1 kB 16.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [129]:
pip install torch



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [134]:
import pandas as pd
import re
import torch
import numpy as np
import transformers
from transformers import BertTokenizer
from torch import nn
from transformers import BertModel
from torch.optim import Adam
from tqdm import tqdm
import nltk


In [145]:
datapath = "/Users/jacklinewambui/Desktop/finalProject/CS72 Project_Emails_training_dataset.csv"
df= pd.read_csv(datapath, encoding="'latin1")
df = df.dropna()
df.isnull().values.any()
df = df.sample(frac=1)
df.head()

ParserError: Error tokenizing data. C error: Expected 1 fields in line 3, saw 2


In [146]:
df.groupby(['Category']).size().plot.bar()

NameError: name 'df' is not defined

In [ ]:
def preprocess_text(sen): 
    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [ ]:
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

In [ ]:
emails = []
sentences = list(df["Emails"])
for sen in sentences:
  emails.append(preprocess_text(sen))

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
labels = {'Related': 0,
          'Unrelated': 1
          }

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [labels[label] for label in df['Category']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['Emails']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [ ]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 5)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [ ]:
np.random.seed(112)
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42), 
                                     [int(.8*len(df)), int(.9*len(df))])

print(len(df_train),len(df_val), len(df_test))
print(type(df_train))

In [ ]:
def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')
                  
EPOCHS = 5
model = BertClassifier()
LR = 1e-6
              
train(model, df_train, df_val, LR, EPOCHS)

In [ ]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    y_preds = []
    y_trues = []

    if use_cuda:
        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)

              y_trues.append(test_label)

              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)
              y_preds.append(output.argmax(dim=1))

              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
    
    # confusion_matrix = metrics.confusion_matrix(y_trues, y_preds, labels=["Related", "Unrelated"])
    # print(confusion_matrix)
    
evaluate(model, df_test)

In [ ]:
def preprocessToken(input_text, tokenizer):
  return tokenizer.encode_plus(input_text, 
                               add_special_tokens = True,
                               max_length = 512,
                               padding='longest',
                               return_attention_mask = True,
                               return_tensors = 'pt'
                               )

In [103]:
def predict(model, userinput):
  input_ids = []
  attention_mask = []
  encoding = preprocessToken(userinput, tokenizer)

  input_ids.append(encoding['input_ids'])
  attention_mask.append(encoding['attention_mask'])
  input_ids = torch.cat(input_ids, dim = 0)
  attention_mask = torch.cat(attention_mask, dim = 0)

  output = model(input_ids, attention_mask)
  prediction = 'Related' if np.argmax(output.detach().numpy()).flatten()==0 else 'Unrelated'
  # print("Tensor is: ", output.argmax(dim=1))
  print("This message is: ", prediction)

  return prediction

GET THE EXTRACTED INFORMATION FROM PRETRAINED STANFORD NER TAGGER 

In [147]:
# python "NER.ipynb"

classified_text= testStanfordNERTTagger(userInput)

NameError: name 'testStanfordNERTTagger' is not defined

Test the BERT Classifier and StanfordNER Model a single email from the gmail account: jackline.w.gathoni.24@dartmouth.edu

In [117]:
import Training

model = Training.preprocess_text(userInput)
pred = predict(model, userInput)


# if(pred == 'Related'):
#   //
#   classified_text = testStanfordNERTTagger(userInput)
#   print(classified_text)

AttributeError: module 'Training' has no attribute 'preprocess_text'

Parse the time from the classified text

In [ ]:
import datetime
from zoneinfo import ZoneInfo
import re

def getTime(classified_text):
  time = []
  specialTimes={}
  specialTimes["noon"]= "12"
  specialTimes["midnight"]= "00"


  for item in classified_text:
    if(item[1] == 'TIME'):
      
      if not any(chr.isdigit() for chr in item[0]): 
        continue

      #regex for parsing time
      # expression results in five groups
      reTime = r"([0-9]*:*[0-9]*)(.*)(-|to)([0-9]*:*[0-9]*)(.*)"
      timeBreakdown = re.search(reTime, item[0], re.IGNORECASE)


      # Group1 = start_time
      # Group2 = am/pm/ other exceptions like noon, midnight
      # Group3 = -/to
      # Group4 = end_time 
      # Group5 = am/pm/other exceptions like noon, midnight
      if timeBreakdown != None:
        startTime = timeBreakdown.group(1)
        startPeriod = timeBreakdown.group(2)
        to = timeBreakdown.group(3)
        endTime = timeBreakdown.group(4)
        endPeriod =  timeBreakdown.group(5)


        if startPeriod in specialTimes:
          startTime = specialTimes[startPeriod]
        if endPeriod in specialTimes:
          endTime = specialTimes[endPeriod]


        def splitTime(time, period):
          timeDiv = time.split(':')
          hours= int(timeDiv[0])
         
          if period == 'p' or period =='pm':
            hours+=12
          if len(timeDiv) == 2:
            minutes = int(timeDiv[1])
          else:
            minutes = 0;
          return hours, minutes
     

        start_time = datetime.time(splitTime(startTime, startPeriod)[0], (splitTime(startTime,startPeriod)[1]), tzinfo = ZoneInfo('US/Eastern'))
        end_time = datetime.time(splitTime(endTime, endPeriod)[0], (splitTime(endTime, endPeriod)[1]), tzinfo = ZoneInfo('US/Eastern'))
        return(start_time, end_time)

  return None

GET CREDENTIALS FROM GOOGLE CALENDAR API

In [112]:
start_time = getTime(classified_text)[0]
end_time = getTime(classified_text)[1]
available = Scheduling.checkAvailability(start_time, end_time)

Getting the upcoming 10 events...


TypeError: combine() argument 2 must be datetime.time, not int

In [ ]:
import re
import Email


# Extracts the sender's email from sender's information: .<(.*)>
reEmail= r".<(.*)>"
service = Email.getService()
emailDetails = Email.getEmails(service)


destination = re.search(reEmail, emailDetails["Sender"], re.IGNORECASE).group(1)
print("destination email is: ", destination)
recipients= re.search(reEmail, emailDetails["Recipients"], re.IGNORECASE)
if recipients:
  recipientEmail = recipients.group(1)
print("other recipient email is: ", recipients)

In [ ]:

#Sends email responding to the sender if available for the time window of the shift dropped

from base64 import urlsafe_b64encode
from email.mime.text import MIMEText


def build_message(destination, obj, body):
    message = MIMEText(body)

    message['to'] = destination
    message['from'] = "jackline.w.gathoni.24@dartmouth.edu"
    message['subject'] = obj
    message['In-Reply-To']= destination
    if recipientEmail != "jackline.w.gathoni.24@dartmouth.edu":
      message['References']= recipients

    return {'raw': urlsafe_b64encode(message.as_bytes()).decode()}

def send_message(service, destination, obj, body):
    return service.users().messages().send(
      userId="me",
      body=build_message(destination, obj, body)
    ).execute()




In [ ]:
if available:
    # test send email
    send_message(service, destination, emailDetails["Subject"], 
                "I can",
    )